In [1]:
import sys
sys.path.insert(0,'..')
from gp3.inference.svi import MFSVI
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
from gp3.kernels.kernels import RBF
import numpy as np
from gp3.likelihoods.likelihoods import Poisson

In [2]:
X_grid_fec = np.genfromtxt('fec_data/X_grid.csv', delimiter = ',')
fec_counts = np.genfromtxt('fec_data/y_hillary.csv', delimiter = ',')
obs_idx_fec = np.genfromtxt('fec_data/obs_idx.csv', delimiter = ',', dtype = np.int32)
trace_fec_counts = go.Scatter3d(x = X_grid_fec[obs_idx_fec,1], y = X_grid_fec[obs_idx_fec,0],
                                z=fec_counts, mode = 'markers', marker=dict(size = 2,))
iplot([trace_fec_counts])

In [4]:
mu = np.ones(X_grid_fec.shape[0])*np.log(np.mean(fec_counts))
kern = RBF(50., 2., 1.)
likelihood = Poisson()

inf_svi = MFSVI(kern, Poisson(), X_grid_fec, fec_counts, obs_idx = obs_idx_fec)
inf_svi.run(2000)

In [5]:
iplot([go.Scatter3d(x = X_grid_fec[obs_idx_fec,1], y = X_grid_fec[obs_idx_fec,0], z=inf_svi.predict()[obs_idx_fec], mode = 'markers', marker=dict(size = 2,), name = 'SVI posterior mean')])